In [48]:
import numpy as np
import os

In [41]:
%pwd

'C:\\Users\\llaur\\OneDrive\\Desktop\\turb\\proj-turbulence\\train'

In [44]:
%cd Downloads/ERA_wind

C:\Users\llaur\Downloads\ERA_wind


In [45]:
fine = np.load("ERA_wind.npz")

# Load

## Train

In [47]:
%cd ../../OneDrive/Desktop/turb/proj-turbulence/train

C:\Users\llaur\OneDrive\Desktop\turb\proj-turbulence\train


In [64]:
u = fine.get('wind_u')
v = fine.get('wind_v')

(744, 4, 121, 201)

In [ ]:
# for lat in range(121)
#     for lon in range(201):
#         # the set of wind speeds at all time stamps at this one point
#         wind_speeds = [np.sqrt((u[i][1][lat][lon])**2 + (v[i][1][lat][lon])**2) for i in range (744)]

List comprehension of wind speeds at level 225

In [67]:
wind_speeds = [[[np.sqrt((u[i][1][lat][lon])**2 + (v[i][1][lat][lon])**2) for lon in range(201)] for lat in range(121)] for i in range(744)]

In [68]:
len(wind_speeds), len(wind_speeds[0]), len(wind_speeds[0][0])

(744, 121, 201)

In [49]:
files = os.listdir(os.getcwd())

Add to coarse and fine lists in order

In [72]:
coarse_train = []
fine_train = []
for file in files: # loop thru files
    if file[0]=="w":
        timestamp = file[19:-4]
        coarse_train.append(np.load(file)) # load file, save into coarse
        fine_train.append(wind_speeds[int(timestamp)]) # load fine version

595 samples, # lat points, # lon points

In [78]:
len(coarse_train), len(coarse_train[0]), len(coarse_train[0][0])

(595, 15, 25)

In [80]:
len(fine_train), len(fine_train[0]), len(fine_train[0][0])

(595, 121, 201)

## Val

In [86]:
%cd ../val

C:\Users\llaur\OneDrive\Desktop\turb\proj-turbulence\val


In [87]:
files = os.listdir(os.getcwd())
coarse_val = []
fine_val = []
for file in files: # loop thru files
    if file[0]=="w":
        timestamp = file[19:-4]
        coarse_val.append(np.load(file)) # load file, save into coarse
        fine_val.append(wind_speeds[int(timestamp)]) # load fine version

In [88]:
len(coarse_val), len(coarse_val[0]), len(coarse_val[0][0])

(75, 15, 25)

In [89]:
len(fine_val), len(fine_val[0]), len(fine_val[0][0])

(75, 121, 201)

## Test

In [90]:
%cd ../test

C:\Users\llaur\OneDrive\Desktop\turb\proj-turbulence\test


In [91]:
files = os.listdir(os.getcwd())
coarse_test = []
fine_test = []
for file in files: # loop thru files
    if file[0]=="w":
        timestamp = file[19:-4]
        coarse_test.append(np.load(file)) # load file, save into coarse
        fine_test.append(wind_speeds[int(timestamp)]) # load fine version

In [92]:
len(coarse_test), len(coarse_test[0]), len(coarse_test[0][0])

(74, 15, 25)

In [93]:
len(fine_test), len(fine_test[0]), len(fine_test[0][0])

(74, 121, 201)

# CNN

In [94]:
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

## Load Data

In [102]:
class PairedImageDataset(Dataset):
    def __init__(self, coarse_images, high_res_images, transform=None):
        self.coarse_images = coarse_images
        self.high_res_images = high_res_images
        self.transform = transform

    def __len__(self):
        return len(self.coarse_images)

    def __getitem__(self, idx):
        coarse_img = self.coarse_images[idx]
        high_res_img = self.high_res_images[idx]

        if self.transform:
            coarse_img = self.transform(coarse_img)
            high_res_img = self.transform(high_res_img)

        return coarse_img, high_res_img

transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])

train = PairedImageDataset(np.array(coarse_train), np.array(fine_train), transform=transform)
test = PairedImageDataset(np.array(coarse_test), np.array(fine_test), transform=transform)
val = PairedImageDataset(np.array(coarse_val), np.array(fine_val), transform=transform)

# load
batch_size = 16
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=True)

## Define CNN

In [103]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 64, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.depth_to_space = nn.PixelShuffle(3)
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.depth_to_space(x)
        return x
    
model = CNN()

## Train CNN

In [104]:
criterion = nn.CrossEntropyLoss() 
optimizer = optim.NAdam(cnn.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    # Validation
    model.eval()
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            val_loss = criterion(outputs, targets)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}')


RuntimeError: Input type (double) and bias type (float) should be the same